In [18]:
import requests
import re
import os
import json
import urllib
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from sklearn import manifold
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
headers = {'Accept-Language': 'en',
           'X-FORWARDED-FOR': '2.21.184.0'}

In [27]:
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2019-12-31&num_votes=1000,&countries=us&languages=en'
response = requests.get(url, headers=headers)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.text)

In [5]:
pics = soup.find_all('img', attrs={'class': 'loadlate'})

In [9]:
pics[0]['loadlate']

'https://m.media-amazon.com/images/M/MV5BOTg4ZTNkZmUtMzNlZi00YmFjLTk1MmUtNWQwNTM0YjcyNTNkXkEyXkFqcGdeQXVyNjg2NjQwMDQ@._V1_UX67_CR0,0,67,98_AL_.jpg'

### Functions to implement
* **[v]** get_link_to_next_page
* **[v]** get_links_to_pics
* **[v]** save_image
* **[v]** download_all_posters

In [10]:
def get_next_page_link(soup):
    links = soup.find_all('a', attrs={'class': 'next-page'})
    if len(links) == 0:
        return None
    else:
        return links[0]['href']

In [13]:
def get_links_to_pics(soup):
    pics_elements = soup.find_all('img', attrs={'class': 'loadlate'})
    pics = [element['loadlate'] for element in pics_elements]
    if len(pics) == 0:
        return None
    return pics

In [24]:
def save_image(pic_url, save_dir=''):
    filename = urllib.parse.urlsplit(pic_url).path.split('/')[-1]
    if not save_dir == '' and not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, filename)
    
    with requests.get(pic_url, stream=True) as r:
        with open(save_path, 'wb') as f:
            r.raise_for_status()
            for chunk in r.iter_content():
                f.write(chunk)

In [29]:
def download_all_posters(url, save_dir='pics', verbose=False):
    headers = {'Accept-Language': 'en',
           'X-FORWARDED-FOR': '2.21.184.0'}
    done = 0
    while True:
        response = requests.get(url, headers=headers)
        assert response.status_code == 200
        soup = BeautifulSoup(response.text)
        pic_urls = get_links_to_pics(soup)
        for pic_url in pic_urls:
            save_image(pic_url, save_dir)
        link = get_next_page_link(soup)
        if verbose:
            done += 1
            print(f'Done with {done} pages')
        if not link:
            break
        url = urllib.parse.urljoin(url, link)
        time.sleep(1)

In [31]:
download_all_posters(url, verbose=True)

Done with 1 pages
Done with 2 pages
Done with 3 pages
Done with 4 pages
Done with 5 pages
Done with 6 pages
Done with 7 pages
Done with 8 pages
Done with 9 pages
Done with 10 pages
Done with 11 pages
Done with 12 pages


In [34]:
from imageio import imread

In [92]:
save_dir = 'pics'
pics = os.listdir(save_dir)

images_orig = []
images_reshaped = []
for pic in pics:
    try:
        image = imread(os.path.join(save_dir, pic))
        images_orig.append(image)
        images_reshaped.append(image.reshape(-1))
    except:
        print(pic)

<ipython-input-92-6dca4d54a27b>:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(os.path.join(save_dir, pic))


In [93]:
images_orig[0].shape

(98, 67, 3)

In [94]:
images_reshaped[0]

Array([231, 219, 193, ..., 228, 216, 192], dtype=uint8)

In [103]:
# images_reshaped_1 = []
# for i in images_reshaped:
#     images_reshaped_1.append(list(i))
# images_reshaped_1

In [80]:
# images_orig = np.array(images_orig)
# images_reshaped = np.array(images_reshaped)

In [81]:
# tsne = manifold.TSNE(n_components=2, init='pca')

In [82]:
# X_tsne = tsne.fit_transform(images_reshaped)

In [83]:
# X_tsne.shape